# Training EfficientNet model on TPU

This notebook trains Efficient model on TPU hardware and also creates a submission file that will be uploaded on kaggle comeptition.

Contents-
1. Introduction to TPU<br>
    1.1. TPU on Kaggle<br>
    1.2 Disadvantages of TPU<br>
2. Train the model
3. Test File Submission
4. References


## 1. Introduction to TPU

A tensor processing unit (TPU) is an AI accelerator application-specific integrated circuit (ASIC) developed by Google specifically for neural network machine learning, particularly using Google's own TensorFlow software. Google began using TPUs internally in 2015, and in 2018 made them available for third party use, both as part of its cloud infrastructure and by offering a smaller version of the chip for sale. **Below is a picture of TPU hardware version3.8** 

<img src="./images/tpu_cores_and_chips.png" width=620 height=620 title="TPU v3-8 hardware, 4 chips, 8 cores" />
At approximately 20 inches (50 cm), a TPU v3-8 board is a fairly sizeable piece of hardware. It sports 4 dual-core TPU chips for a total of 8 TPU cores.

Each TPU core has a traditional vector processing part (VPU) as well as dedicated matrix multiplication hardware capable of processing 128x128 matrices. This is the part that specifically accelerates machine learning workloads.

TPUs are equipped with 128GB of high-speed memory allowing larger batches, larger models and also larger training inputs. In the sample above, you can try using 512x512 px input images, also provided in the dataset, and see the TPU v3-8 handle them easily.

Compared to a graphics processing unit, it is designed for a high volume of low precision computation (e.g. as little as 8-bit precision) with more input/output operations per joule, and lacks hardware for rasterisation/texture mapping. The TPU ASICs are mounted in a heatsink assembly, which can fit in a hard drive slot within a data center rack.

### 1.1. TPU on Kaggle
**Google doesnot sell the TPUs commercially.** But, they announced that the second gen of the TPUs will be accessible through the Google cloud computing platform. Basically you have to rent it, similar to the Amazon's AWS (Amazon machine learning platform).
TPUs are now available on Kaggle, for free. They are supported in Tensorflow 2.1 both through the Keras high-level API and, at a lower level, in models using a custom training loop.

In our case, we will be using the free quota of TPU available on Kaggle.

### 1.2 Disadvantages of TPU
There are a few drawbacks to be aware of:

 - The topology is unlike other hardware platforms and is not trivial to work with for those not familiar with DevOps and the idiosyncrasies of the TPU itself
 - The TPU only supports TensorFlow currently, although other frameworks may be supported in the future   
 - Certain TensorFlow operations (e.g. customer operations written in C++) are not supported
 - TPUs are optimal for large models with very large batch sizes and workloads that are dominated by matrix-multiplication. Models dominated by algebra will not perform well.
 

# 2. Train the model

In this section we will,
- load the data 
- intialize the TPU
- train EfficientNet model.

The below code installs the necessary library
## 2.1 Steps to setup TPU in kaggle/Google notebook
Once you have flipped the "Accelerator" switch in your notebook to "TPU v3-8", this is how to enable TPU training in Tensorflow Keras:
1. TPUs are network-connected accelerators and you must first locate them on the network. This is what _TPUClusterResolver()_ does.

2. Two additional lines of boilerplate are required to define a _TPUStrategy_. This object contains the necessary distributed training code that will work on TPUs with their 8 compute cores.

3. Finally, use the _TPUStrategy_ by instantiating your model in the scope of the strategy. This creates the model on the TPU. Model size is constrained by the TPU RAM only, not by the amount of memory available on the VM running your Python code. Model creation and model training use the usual Keras APIs.

In [2]:
!pip install -q efficientnet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


## 2.2 Load data
The below code does the following:
1. imports necessary libraries
2. locate the TPU on the network
3. connect to the searched TPU
4. intialize the tpu environment

In [4]:
import os
import re
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
import dill
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.master())

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  grpc://10.0.0.2:8470


From the above output we can see that TPU address is grpc://10.0.0.2:8470. Now, the next step is to load the dataset and set up the model's hyperparameters. The below code does the following jobs

1. Set the prefetch strategy as AUTOTUNE. Prefetching overlaps the preprocessing and model execution of a training step. It can be used to decouple the time when data is produced from the time when data is consumed. In particular, the transformation uses a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested. The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step. But,to tf.data.experimental.AUTOTUNE which will help the runtime to tune the value dynamically at runtime.
2. epochs = 15, learning rate = 1e-3, input image size as 384 X 384.
3. fetch the training & test records in 'tfrec' format. Also fetch the submission csv.

In [5]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('melanoma-384x384')

# Configuration
EPOCHS = 15
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
AUG_BATCH = BATCH_SIZE
IMAGE_SIZE = [384, 384]
# Seed
SEED = 333
# Learning rate
LR = 1e-3
# cutmix prob
cutmix_rate = 0.30

# training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
# test filenames directory
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')
# submission file
submission = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

The below helper functions are useful in seeding the environment, load the datasets in batches and convert the byte images into jpeg and normalize the pixel values.

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

def load_dataset_full(filenames):        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # returns a dataset of (image_name, target)
    dataset = dataset.map(read_tfrecord_full, num_parallel_calls = AUTO) 
    return dataset

def get_data_full(filenames):
    dataset = load_dataset_full(filenames)
    dataset = dataset.map(setup_input3, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0 
    # explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image


the below functions are helpful in combining the metadata and the images into single TF record that is later input to the model. This also converts the variable 'anatom_site_general_challenge' to categorical values.
This is done for both Train and Test images.

In [ ]:
# function for training and validation dataset
def setup_input1(image, label, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, label

# function for the test set
def setup_input2(image, image_name, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name

# function for the validation (image name)
def setup_input3(image, image_name, target, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name, target


The below 3 functions input the dataset to the model by batches diregarding data order. Order does not matter since we will be shuffling the data anyway. They also prefetch the next batch of samples while training the current sample to fasten the training process. 

In [ ]:
def get_training_dataset(filenames, labeled = True, ordered = False):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(transform, num_parallel_calls = AUTO)
    # the training dataset must repeat for several epochs
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(filenames, labeled = True, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # using gpu, not enought memory to use cache
    # dataset = dataset.cache()
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(filenames, labeled = False, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input2, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

  
def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

The below functions returns a pair of a train/test sample in the form (image, label, images' metadata).

In [ ]:
# this function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.float32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, label, data)
    return image, label, data

# this function parse our image and also get our image_name (id) to perform predictions
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "image_name": tf.io.FixedLenFeature([], tf.string),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, key, data)
    return image, image_name, data

The below functions returns a pair of a train/test sample in the form (image, image name,label, images' metadata). We also have another helper function to do data augmentation. Following are the augmentations done.
1. randomly flip image left/right & up/down
2. randomly set hue
3. randomly set saturation
4. randomly set contrast
5. randomly set brightness

In [ ]:
def data_augment(data, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement 
    # in the next function (below), this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    data['inp1'] = tf.image.random_flip_left_right(data['inp1'])
    data['inp1'] = tf.image.random_flip_up_down(data['inp1'])
    data['inp1'] = tf.image.random_hue(data['inp1'], 0.01)
    data['inp1'] = tf.image.random_saturation(data['inp1'], 0.7, 1.3)
    data['inp1'] = tf.image.random_contrast(data['inp1'], 0.8, 1.2)
    data['inp1'] = tf.image.random_brightness(data['inp1'], 0.1)
    
    return data, label


# this function parse our images and also get the target variable
def read_tfrecord_full(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "image_name": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64), 
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    target = tf.cast(example['target'], tf.float32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    return image, image_name, target, data

The below function returns the transformation matrix that is to be applied on the batches of train samples.

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

The below helper functions applies the transformation matrix on the entire batch of train samples. The 3rd data augmentation strategy is called "data CutMix augmentation strategy" where patches are cut and pasted among training images. 

In the CutMix algorithm, for each image of batch a random region (in our case, a random bounding box region) is replaced with a patch from another training image.
The parameter lambda (the variable p in the code) that determines the size of the bounding box is stochastically sampled from a a distribution.
The label of each resultant augmented image is estimated as a weighted sum of the original label and the label of the image from which the modified patch is borrowed.

This technique has been proven to be effective for guiding the model to attend on less discriminative parts of objects (e.g. leg as opposed to head of a person), thereby letting the network generalize better and have better object localization capabilities

In [6]:
def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        rot = 15. * tf.random.normal([1],dtype='float32')
    else:
        rot = 180. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        shr = 5. * tf.random.normal([1],dtype='float32') 
    else:
        shr = 2. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10. 
    else:
        h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/8.
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10. 
    else:
        w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/8.
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        h_shift = 16. * tf.random.normal([1],dtype='float32') 
    else:
        h_shift = 8. * tf.random.normal([1],dtype='float32')
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        w_shift = 16. * tf.random.normal([1],dtype='float32') 
    else:
        w_shift = 8. * tf.random.normal([1],dtype='float32')
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image['inp1'],tf.transpose(idx3))
        
    return {'inp1': tf.reshape(d,[DIM,DIM,3]), 'inp2': image['inp2']}, label

# function to apply cutmix augmentation
def cutmix(image, label):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    
    DIM = IMAGE_SIZE[0]    
    imgs = []; labs = []
    
    for j in range(BATCH_SIZE):
        
        #random_uniform( shape, minval=0, maxval=None)        
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([], 0, 1) <= cutmix_rate, tf.int32)
        
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast(tf.random.uniform([], 0, BATCH_SIZE), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        
        # Beta(1, 1)
        b = tf.random.uniform([], 0, 1) # this is beta dist with alpha=1.0
        

        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image['inp1'][j,ya:yb,0:xa,:]
        two = image['inp1'][k,ya:yb,xa:xb,:]
        three = image['inp1'][j,ya:yb,xb:DIM,:]        
        #ya:yb
        middle = tf.concat([one,two,three],axis=1)

        img = tf.concat([image['inp1'][j,0:ya,:,:],middle,image['inp1'][j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE, 1))
    return {'inp1': image2, 'inp2': image['inp2']}, label2


# function to count how many photos we have in
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)


NUM_TRAINING_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.8)
# use validation data for training
NUM_VALIDATION_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.2)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Dataset: 26153 training images, 6538 validation images, 10982 unlabeled test images


From the above output we can see that the correct number of train, val and test images are loaded. Now, we will define a function to define a new type of Loss Function called "Binary Focal Loss".

Focal Loss is binary cross-entropy by introducing a hyperparameter γ (gamma), called the focusing parameter. This allows hard-to-classify examples to be penalized more heavily relative to easy-to-classify examples. The focal loss is defined as
\begin{align}L(y, \hat{p})
= -\alpha y \left(1 - \hat{p}\right)^\gamma \log(\hat{p})
- (1 - y) \hat{p}^\gamma \log(1 - \hat{p})\end{align} Where,
- $ y \in \{0, 1\} $ is a binary class label,
- $ \hat{p} \in [0, 1 $  is an estimate of the probability of the positive class,
- $ \gamma $  is the focusing parameter that specifies how much higher-confidence correct predictions contribute to the overall loss (the higher the γ, the higher the rate at which easy-to-classify examples are down-weighted). We have chosen this as 2.
- $ \alpha $  is a hyperparameter that governs the trade-off between precision and recall by weighting errors for the positive class up or down ($ \alpha $  is the default, which is the same as no weighting). We have chosen this as .8 

The usual weighted binary cross-entropy loss is recovered by setting $ \gamma = 0 $ 

In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

The below code modifies the EfficientNetB3 to suit our Binary Classification by adding 4 Dense Layers and adding "sigmoid" activation for the output layer. Following are the model hyperparameters used.
1. Adam Optimizer
2. Metrics to be tracked is BinaryAccuracy and AUC
3. focal_loss with gamma = 2.0 & alpha = 0.80

In [ ]:
def get_model():
    
    
    with strategy.scope():
        inp1 = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        inp2 = tf.keras.layers.Input(shape = (9), name = 'inp2')
        efnetb3 = efn.EfficientNetB3(weights = 'imagenet', include_top = False)
        x = efnetb3(inp1)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x1 = tf.keras.layers.Dense(50)(inp2)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Activation('relu')(x1)
        concat = tf.keras.layers.concatenate([x, x1])
        concat = tf.keras.layers.Dense(512, activation = 'relu')(concat)
        concat = tf.keras.layers.BatchNormalization()(concat)
        concat = tf.keras.layers.Dropout(0.2)(concat)
        concat = tf.keras.layers.Dense(182, activation = 'relu')(concat)
        concat = tf.keras.layers.BatchNormalization()(concat)
        concat = tf.keras.layers.Dropout(0.3)(concat)
        output = tf.keras.layers.Dense(1, activation = 'sigmoid')(concat)

        model = tf.keras.models.Model(inputs = [inp1, inp2], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)
        # opt = tfa.optimizers.SWA(opt)

        model.compile(
            optimizer = opt,
            loss = [binary_focal_loss(gamma = 2.0, alpha = 0.80)],
            metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()]
        )

        return model

The below code Trains the model and also predicts on Test images.

1. Split the data into 5 folds
2. For each of the fold, train a EfficientNetB3
3. Predict 5 sets of predictions for Test using these 5 models and take their average as the final prediction. This technique is called Out of Fold (OOF) technique.

In [7]:
def train_and_predict(SUB, folds = 5):
    
    models = []
    oof_image_name = []
    oof_target = []
    oof_prediction = []
    
    # seed everything
    seed_everything(SEED)

    kfold = KFold(folds, shuffle = True, random_state = SEED)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        train_dataset = get_training_dataset([TRAINING_FILENAMES[x] for x in trn_ind], labeled = True, ordered = False)
        val_dataset = get_validation_dataset([TRAINING_FILENAMES[x] for x in val_ind], labeled = True, ordered = True)
        K.clear_session()
        model = get_model()
        # using early stopping using val loss
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', mode = 'max', patience = 5, 
                                                      verbose = 1, min_delta = 0.0001, restore_best_weights = True)
        # lr scheduler
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'max')
        history = model.fit(train_dataset, 
                            steps_per_epoch = STEPS_PER_EPOCH,
                            epochs = EPOCHS,
                            callbacks = [early_stopping, cb_lr_schedule],
                            validation_data = val_dataset,
                            verbose = 2)
        models.append(model)
        
        # want to predict the validation set and save them for stacking
        number_of_files = count_data_items([TRAINING_FILENAMES[x] for x in val_ind])
        dataset = get_data_full([TRAINING_FILENAMES[x] for x in val_ind])
        # get the image name
        image_name = dataset.map(lambda image, image_name, target: image_name).unbatch()
        image_name = next(iter(image_name.batch(number_of_files))).numpy().astype('U')
        # get the real target
        target = dataset.map(lambda image, image_name, target: target).unbatch()
        target = next(iter(target.batch(number_of_files))).numpy()
        # predict the validation set
        image = dataset.map(lambda image, image_name, target: image)
        probabilities = model.predict(image)
        oof_image_name.extend(list(image_name))
        oof_target.extend(list(target))
        oof_prediction.extend(list(np.concatenate(probabilities)))
    
    print('\n')
    print('-'*50)
    # save oof predictions
    oof_df = pd.DataFrame({'image_name': oof_image_name, 'target': oof_target, 'predictions': oof_prediction})
    oof_df.to_csv('EfficientNetB3_384.csv', index = False)
        
    # since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_ds = get_test_dataset(TEST_FILENAMES, labeled = False, ordered = True)
    test_images_ds = test_ds.map(lambda image, image_name: image)
    
    print('Computing predictions...')
    probabilities = np.average([np.concatenate(models[i].predict(test_images_ds)) for i in range(folds)], axis = 0)
    print('Generating submission.csv file...')
    test_ids_ds = test_ds.map(lambda image, image_name: image_name).unbatch()
    # all in one batch
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
    pred_df = pd.DataFrame({'image_name': test_ids, 'target': probabilities})
    SUB.drop('target', inplace = True, axis = 1)
    SUB = SUB.merge(pred_df, on = 'image_name')
    SUB.to_csv('submission.csv', index = False)
    print('Submission file generated')
    
    return oof_target, oof_prediction
    
oof_target, oof_prediction = train_and_predict(submission)



--------------------------------------------------
Training fold 1
44113920/44107200 [==============================] - 2s 0us/step
Epoch 1/15
204/204 - 74s - loss: 0.5151 - auc: 0.5603 - binary_accuracy: 0.8228 - val_loss: 0.2242 - val_auc: 0.7200 - val_binary_accuracy: 0.9821 - lr: 0.0010
Epoch 2/15
204/204 - 56s - loss: 0.2826 - auc: 0.6305 - binary_accuracy: 0.9524 - val_loss: 1.1960 - val_auc: 0.6625 - val_binary_accuracy: 0.9777 - lr: 0.0010
Epoch 3/15
204/204 - 58s - loss: 0.2484 - auc: 0.6827 - binary_accuracy: 0.9688 - val_loss: 0.2825 - val_auc: 0.7213 - val_binary_accuracy: 0.9352 - lr: 0.0010
Epoch 4/15
204/204 - 60s - loss: 0.2184 - auc: 0.7386 - binary_accuracy: 0.9743 - val_loss: 0.2024 - val_auc: 0.8273 - val_binary_accuracy: 0.9818 - lr: 0.0010
Epoch 5/15
204/204 - 57s - loss: 0.2295 - auc: 0.7313 - binary_accuracy: 0.9733 - val_loss: 0.2034 - val_auc: 0.7657 - val_binary_accuracy: 0.9824 - lr: 0.0010
Epoch 6/15

Epoch 00006: ReduceLROnPlateau reducing learning rate 

Epoch 15/15
204/204 - 58s - loss: 0.1865 - auc: 0.8311 - binary_accuracy: 0.9739 - val_loss: 0.1726 - val_auc: 0.8671 - val_binary_accuracy: 0.9807 - lr: 1.6000e-04


--------------------------------------------------
Training fold 4
Epoch 1/15
204/204 - 79s - loss: 0.5070 - auc: 0.6244 - binary_accuracy: 0.8304 - val_loss: 0.3025 - val_auc: 0.7230 - val_binary_accuracy: 0.9127 - lr: 0.0010
Epoch 2/15
204/204 - 56s - loss: 0.2935 - auc: 0.6382 - binary_accuracy: 0.9613 - val_loss: 0.2711 - val_auc: 0.5064 - val_binary_accuracy: 0.9816 - lr: 0.0010
Epoch 3/15

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
204/204 - 56s - loss: 0.2425 - auc: 0.7099 - binary_accuracy: 0.9702 - val_loss: 0.2936 - val_auc: 0.6127 - val_binary_accuracy: 0.9823 - lr: 0.0010
Epoch 4/15
204/204 - 59s - loss: 0.2226 - auc: 0.7354 - binary_accuracy: 0.9750 - val_loss: 0.1950 - val_auc: 0.8245 - val_binary_accuracy: 0.9823 - lr: 4.0000e-04
Epoch 5/15
204/204 - 58s - loss: 0.2042 -

## 3. Submission
The below code creates submission csv file that will be uploaded on kaggle. **This model generated public score=0.8856**

In [8]:
submission = pd.read_csv('submission_tpu_effnet.csv')
submission.head()

,image_name,target
0,ISIC_0052060,0.164781
1,ISIC_0052349,0.114981
2,ISIC_0058510,0.101569
3,ISIC_0073313,0.079869
4,ISIC_0073502,0.258102


## 4. References

1. https://en.wikipedia.org/wiki/Tensor_processing_unit
2. Cutmix paper https://arxiv.org/abs/1905.04899